## Data Scrapping

In [8]:
import praw
import pandas as pd
from datetime import datetime
import time
import os

# --- SETUP REDDIT AUTH ---
reddit = praw.Reddit(
    client_id='trvEAgsIJjIc4SpF1kASCg',
    client_secret='QRgVLo1_3h1B02PINYv5uUcc1hK9aA',
    user_agent='mental-health-resproj',
    username='kohor_',
    password='Sq5dS$*T7#8%gua92o@9'
)

# --- CONFIG ---
subreddits = [
    'depression',
    'depression_help',
    'anxiety',
    'mentalhealth',
    'SuicideWatch',
    'DecidingToBeBetter',
    'BipolarReddit',
    'OCD',
    'ADHD',
    'therapy',
    'StopSelfHarm',
    'selfhelp',
    'socialanxiety'
]

start_year = 2015
start_timestamp = int(datetime(start_year, 1, 1).timestamp())

post_data = []
comment_data = []

# --- SCRAPE ---
for sub in subreddits:
    print(f"🔎 Scraping r/{sub}...")
    try:
        for post in reddit.subreddit(sub).new(limit=None):  # Get as many as allowed
            if post.created_utc < start_timestamp:
                continue

            post_author = post.author.name if post.author else None

            # Store post
            post_data.append({
                'subreddit': sub,
                'id': post.id,
                'title': post.title,
                'text': post.selftext,
                'author': post_author,
                'score': post.score,
                'num_comments': post.num_comments,
                'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'url': post.url
            })

            # Load comments
            post.comments.replace_more(limit=0)
            for top_comment in post.comments:
                comment_author = top_comment.author.name if top_comment.author else None
                comment_entry = {
                    'post_id': post.id,
                    'comment_id': top_comment.id,
                    'comment_author': comment_author,
                    'comment_body': top_comment.body,
                    'comment_score': top_comment.score,
                    'comment_created': datetime.utcfromtimestamp(top_comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    'author_reply': None,
                    'author_reply_id': None
                }

                # Search for author reply to this comment
                for reply in top_comment.replies:
                    if reply.author and reply.author.name == post_author:
                        comment_entry['author_reply'] = reply.body
                        comment_entry['author_reply_id'] = reply.id
                        break  # First author reply is enough

                comment_data.append(comment_entry)

            # Optional: be nice to Reddit API
            time.sleep(1)

    except Exception as e:
        print(f"⚠️ Error scraping r/{sub}: {e}")
        continue

# --- SAVE ---
os.makedirs("data", exist_ok=True)

# Save posts
df_posts = pd.DataFrame(post_data)
df_posts.to_csv("data/reddit_posts_mentalhealth_2015_present.csv", index=False)

# Save comments + replies
df_comments = pd.DataFrame(comment_data)
df_comments.to_csv("data/reddit_comments_with_author_replies.csv", index=False)

print(f"\n✅ Scraped {len(df_posts)} posts and {len(df_comments)} comments with potential author replies.")

🔎 Scraping r/depression...


/var/folders/jm/36rrr53s1w388ms7208drrcm0000gn/T/ipykernel_21099/4174835129.py:58: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


KeyboardInterrupt: 

In [6]:
import praw
import pandas as pd
from datetime import datetime
import os

# --- SETUP REDDIT AUTH ---
reddit = praw.Reddit(
    client_id='trvEAgsIJjIc4SpF1kASCg',
    client_secret='QRgVLo1_3h1B02PINYv5uUcc1kK9aA',
    user_agent='mental-health-resproj',
    username='kohor_',
    password='Sq5dS$*T7#8%gua92o@9'
)

# --- CONFIG ---
subreddits = [
    'depression',
    'depression_help',
    'anxiety',
    'mentalhealth',
    'SuicideWatch',
    'DecidingToBeBetter',
    'BipolarReddit',
    'OCD',
    'ADHD',
    'therapy',
    'StopSelfHarm',
    'selfhelp',
    'socialanxiety'
]

post_limit = 2000

post_data = []
comment_data = []

# --- SCRAPE ---
for sub in subreddits:
    print(f"🔎 Scraping r/{sub} (up to {post_limit} posts)...")
    try:
        for post in reddit.subreddit(sub).new(limit=post_limit):
            post_author = post.author.name if post.author else None

            # Store post info
            post_data.append({
                'subreddit': sub,
                'post_id': post.id,
                'title': post.title,
                'text': post.selftext,
                'author': post_author,
                'score': post.score,
                'num_comments': post.num_comments,
                'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'url': post.url
            })

            # Load comments
            post.comments.replace_more(limit=0)
            for top_comment in post.comments:
                comment_author = top_comment.author.name if top_comment.author else None
                comment_entry = {
                    'post_id': post.id,
                    'comment_id': top_comment.id,
                    'comment_author': comment_author,
                    'comment_body': top_comment.body,
                    'comment_score': top_comment.score,
                    'comment_created': datetime.utcfromtimestamp(top_comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    'author_reply': None,
                    'author_reply_id': None
                }

                # Check if post author replied to this comment
                for reply in top_comment.replies:
                    if reply.author and reply.author.name == post_author:
                        comment_entry['author_reply'] = reply.body
                        comment_entry['author_reply_id'] = reply.id
                        break  # only first author reply

                comment_data.append(comment_entry)

    except Exception as e:
        print(f"⚠️ Error scraping r/{sub}: {e}")
        continue

# --- SAVE TO CSV ---
os.makedirs("data", exist_ok=True)

df_posts = pd.DataFrame(post_data)
df_posts.to_csv("data/reddit_data_posts.csv", index=False)

df_comments = pd.DataFrame(comment_data)
df_comments.to_csv("data/reddit_data_comments_with_author_replies.csv", index=False)

print(f"\n✅ Done! {len(df_posts)} posts and {len(df_comments)} comments (with author replies where available) saved.")

🔎 Scraping r/depression (up to 2000 posts)...
⚠️ Error scraping r/depression: received 401 HTTP response
🔎 Scraping r/depression_help (up to 2000 posts)...
⚠️ Error scraping r/depression_help: received 401 HTTP response
🔎 Scraping r/anxiety (up to 2000 posts)...
⚠️ Error scraping r/anxiety: received 401 HTTP response
🔎 Scraping r/mentalhealth (up to 2000 posts)...
⚠️ Error scraping r/mentalhealth: received 401 HTTP response
🔎 Scraping r/SuicideWatch (up to 2000 posts)...
⚠️ Error scraping r/SuicideWatch: received 401 HTTP response
🔎 Scraping r/DecidingToBeBetter (up to 2000 posts)...
⚠️ Error scraping r/DecidingToBeBetter: received 401 HTTP response
🔎 Scraping r/BipolarReddit (up to 2000 posts)...
⚠️ Error scraping r/BipolarReddit: received 401 HTTP response
🔎 Scraping r/OCD (up to 2000 posts)...
⚠️ Error scraping r/OCD: received 401 HTTP response
🔎 Scraping r/ADHD (up to 2000 posts)...
⚠️ Error scraping r/ADHD: received 401 HTTP response
🔎 Scraping r/therapy (up to 2000 posts)...
⚠️ E

In [7]:
import praw
import pandas as pd
from datetime import datetime
import os
import time

# --- SETUP REDDIT AUTH ---
reddit = praw.Reddit(
    client_id='trvEAgsIJjIc4SpF1kASCg',
    client_secret='QRgVLo1_3h1B02PINYv5uUcc1kK9aA',
    user_agent='mental-health-resproj',
    username='kohor_',
    password='Sq5dS$*T7#8%gua92o@9'
)

# --- CONFIG ---
subreddits = [
    'depression', 'depression_help', 'anxiety', 'mentalhealth',
    'SuicideWatch', 'DecidingToBeBetter', 'BipolarReddit',
    'OCD', 'ADHD', 'therapy', 'StopSelfHarm', 'selfhelp', 'socialanxiety'
]

target_limit = 2000  # total posts per subreddit
chunk_size = 200     # posts per request
sleep_secs = 2       # delay between chunks

post_data = []
comment_data = []

for sub in subreddits:
    print(f"\n🔎 Scraping r/{sub} (target: {target_limit} posts)...")
    count = 0
    try:
        submissions = reddit.subreddit(sub).new(limit=None)
        for post in submissions:
            if count >= target_limit:
                break

            post_author = post.author.name if post.author else None
            post_data.append({
                'subreddit': sub,
                'post_id': post.id,
                'title': post.title,
                'text': post.selftext,
                'author': post_author,
                'score': post.score,
                'num_comments': post.num_comments,
                'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'url': post.url
            })

            try:
                post.comments.replace_more(limit=0)
                for top_comment in post.comments:
                    comment_author = top_comment.author.name if top_comment.author else None
                    comment_entry = {
                        'post_id': post.id,
                        'comment_id': top_comment.id,
                        'comment_author': comment_author,
                        'comment_body': top_comment.body,
                        'comment_score': top_comment.score,
                        'comment_created': datetime.utcfromtimestamp(top_comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                        'author_reply': None,
                        'author_reply_id': None
                    }

                    for reply in top_comment.replies:
                        if reply.author and reply.author.name == post_author:
                            comment_entry['author_reply'] = reply.body
                            comment_entry['author_reply_id'] = reply.id
                            break

                    comment_data.append(comment_entry)
            except Exception as ce:
                print(f"⚠️ Error loading comments for post {post.id}: {ce}")

            count += 1

            if count % chunk_size == 0:
                print(f"  ...scraped {count} posts from r/{sub} so far")
                time.sleep(sleep_secs)

    except Exception as e:
        print(f"❌ Error scraping r/{sub}: {e}")
        continue

# --- SAVE TO CSV ---
os.makedirs("data", exist_ok=True)
df_posts = pd.DataFrame(post_data)
df_posts.to_csv("data/reddit_data_posts.csv", index=False)

df_comments = pd.DataFrame(comment_data)
df_comments.to_csv("data/reddit_data_comments_with_author_replies.csv", index=False)

print(f"\n✅ Done! {len(df_posts)} posts and {len(df_comments)} comments saved.")


🔎 Scraping r/depression (target: 2000 posts)...
❌ Error scraping r/depression: received 401 HTTP response

🔎 Scraping r/depression_help (target: 2000 posts)...
❌ Error scraping r/depression_help: received 401 HTTP response

🔎 Scraping r/anxiety (target: 2000 posts)...
❌ Error scraping r/anxiety: received 401 HTTP response

🔎 Scraping r/mentalhealth (target: 2000 posts)...
❌ Error scraping r/mentalhealth: received 401 HTTP response

🔎 Scraping r/SuicideWatch (target: 2000 posts)...
❌ Error scraping r/SuicideWatch: received 401 HTTP response

🔎 Scraping r/DecidingToBeBetter (target: 2000 posts)...
❌ Error scraping r/DecidingToBeBetter: received 401 HTTP response

🔎 Scraping r/BipolarReddit (target: 2000 posts)...
❌ Error scraping r/BipolarReddit: received 401 HTTP response

🔎 Scraping r/OCD (target: 2000 posts)...
❌ Error scraping r/OCD: received 401 HTTP response

🔎 Scraping r/ADHD (target: 2000 posts)...
❌ Error scraping r/ADHD: received 401 HTTP response

🔎 Scraping r/therapy (target

In [9]:
import praw
import pandas as pd
from datetime import datetime
import time
import os

# --- SETUP REDDIT AUTH ---
reddit = praw.Reddit(
    client_id='trvEAgsIJjIc4SpF1kASCg',
    client_secret='QRgVLo1_3h1B02PINYv5uUcc1hK9aA',
    user_agent='mental-health-resproj',
    username='kohor_',
    password='Sq5dS$*T7#8%gua92o@9'
)

# --- CONFIG ---
subreddits = [
    'depression', 'depression_help', 'anxiety', 'mentalhealth',
    'SuicideWatch', 'DecidingToBeBetter', 'BipolarReddit',
    'OCD', 'ADHD', 'therapy', 'StopSelfHarm', 'selfhelp', 'socialanxiety'
]

start_year = 2015
start_timestamp = int(datetime(start_year, 1, 1).timestamp())

post_limit = 500  # sensible number per subreddit

post_data = []
comment_data = []

# --- SCRAPE ---
for sub in subreddits:
    print(f"🔎 Scraping top posts from r/{sub}...")
    try:
        for post in reddit.subreddit(sub).top(limit=post_limit):
            if post.created_utc < start_timestamp:
                continue  # skip old posts

            post_author = post.author.name if post.author else None

            # Save post
            post_data.append({
                'subreddit': sub,
                'id': post.id,
                'title': post.title,
                'text': post.selftext,
                'author': post_author,
                'score': post.score,
                'num_comments': post.num_comments,
                'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'url': post.url
            })

            # Comments and author replies
            try:
                post.comments.replace_more(limit=0)
                for top_comment in post.comments:
                    comment_author = top_comment.author.name if top_comment.author else None
                    comment_entry = {
                        'post_id': post.id,
                        'comment_id': top_comment.id,
                        'comment_author': comment_author,
                        'comment_body': top_comment.body,
                        'comment_score': top_comment.score,
                        'comment_created': datetime.utcfromtimestamp(top_comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                        'author_reply': None,
                        'author_reply_id': None
                    }

                    for reply in top_comment.replies:
                        if reply.author and reply.author.name == post_author:
                            comment_entry['author_reply'] = reply.body
                            comment_entry['author_reply_id'] = reply.id
                            break

                    comment_data.append(comment_entry)
            except Exception as ce:
                print(f"⚠️ Comment error in post {post.id}: {ce}")

            time.sleep(0.5)  # polite delay

    except Exception as e:
        print(f"❌ Error scraping r/{sub}: {e}")
        continue

# --- SAVE ---
os.makedirs("data", exist_ok=True)

df_posts = pd.DataFrame(post_data)
df_posts.to_csv("data/reddit_posts_top500.csv", index=False)

df_comments = pd.DataFrame(comment_data)
df_comments.to_csv("data/reddit_comments_top500.csv", index=False)

print(f"\n✅ Finished! {len(df_posts)} posts and {len(df_comments)} comments saved.")

🔎 Scraping top posts from r/depression...


/var/folders/jm/36rrr53s1w388ms7208drrcm0000gn/T/ipykernel_21099/2715727796.py:50: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
/var/folders/jm/36rrr53s1w388ms7208drrcm0000gn/T/ipykernel_21099/2715727796.py:65: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'comment_created': datetime.utcfromtimestamp(top_comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


KeyboardInterrupt: 

In [10]:
import praw
import pandas as pd
from datetime import datetime
import os

# --- SETUP REDDIT AUTH ---
reddit = praw.Reddit(
    client_id='trvEAgsIJjIc4SpF1kASCg',
    client_secret='QRgVLo1_3h1B02PINYv5uUcc1hK9aA',
    user_agent='mental-health-resproj',
    username='kohor_',
    password='Sq5dS$*T7#8%gua92o@9'
)

# --- CONFIG ---
subreddits = [
    'depression', 'depression_help', 'anxiety', 'mentalhealth',
    'SuicideWatch', 'DecidingToBeBetter', 'BipolarReddit',
    'OCD', 'ADHD', 'therapy', 'StopSelfHarm', 'selfhelp', 'socialanxiety'
]

post_limit = 500  # number of top posts per subreddit
post_data = []

# --- SCRAPE POSTS ONLY ---
for sub in subreddits:
    print(f"🔎 Scraping top {post_limit} posts from r/{sub}...")
    try:
        for post in reddit.subreddit(sub).top(limit=post_limit):
            post_data.append({
                'subreddit': sub,
                'id': post.id,
                'title': post.title,
                'text': post.selftext,
                'author': post.author.name if post.author else None,
                'score': post.score,
                'num_comments': post.num_comments,
                'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'url': post.url
            })
    except Exception as e:
        print(f"❌ Error scraping r/{sub}: {e}")
        continue

# --- SAVE ---
os.makedirs("data", exist_ok=True)
df = pd.DataFrame(post_data)
df.to_csv("data/reddit_top500_posts_per_subreddit.csv", index=False)

print(f"\n✅ Done! Scraped {len(df)} posts. Saved to: data/reddit_top500_posts_per_subreddit.csv")

🔎 Scraping top 500 posts from r/depression...


/var/folders/jm/36rrr53s1w388ms7208drrcm0000gn/T/ipykernel_21099/2459199829.py:38: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


🔎 Scraping top 500 posts from r/depression_help...
🔎 Scraping top 500 posts from r/anxiety...
🔎 Scraping top 500 posts from r/mentalhealth...
🔎 Scraping top 500 posts from r/SuicideWatch...
🔎 Scraping top 500 posts from r/DecidingToBeBetter...
🔎 Scraping top 500 posts from r/BipolarReddit...
🔎 Scraping top 500 posts from r/OCD...
🔎 Scraping top 500 posts from r/ADHD...
🔎 Scraping top 500 posts from r/therapy...
🔎 Scraping top 500 posts from r/StopSelfHarm...
❌ Error scraping r/StopSelfHarm: received 403 HTTP response
🔎 Scraping top 500 posts from r/selfhelp...
🔎 Scraping top 500 posts from r/socialanxiety...

✅ Done! Scraped 6000 posts. Saved to: data/reddit_top500_posts_per_subreddit.csv
